# Análise de Livros e Avaliaçõe

* Maria Sofia Abreu
* Maio de 2025

## Objetivos do Estudo

Neste estudo, o objetivo é analisar um banco de dados relacionado a livros, autores, editoras, avaliações e revisões. A análise buscará identificar padrões e insights importantes sobre o comportamento dos leitores, editoras e autores. Vamos realizar consultas SQL para obter informações detalhadas sobre o número de livros publicados, avaliações de usuários, e destacar livros e autores mais populares, entre outros aspectos.

## Estudo das Tabelas (Imprimir as Primeiras Linhas)

In [11]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

db_config = {
    'user': 'practicum_student',  # username
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',  # password
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,  # connection port
    'db': 'data-analyst-final-project-db'  # the name of the database
}


connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)


engine = create_engine(connection_string, connect_args={'sslmode': 'require'})

In [13]:
# pd.io.sql.read_sql(query, con = engine)

## Realizando as Consultas SQL para Cada Tarefa

### Encontrar o número de livros lançados depois de 1 de janeiro de 2000

In [27]:
def task_1():
    query = '''
    SELECT COUNT(*) AS books_after_2000
    FROM books
    WHERE publication_date > '2000-01-01';
    '''
    result = pd.read_sql(query, con=engine)
    return result

# Executando a tarefa 1 e mostrando o resultado
result_1 = task_1()
print(result_1)

   books_after_2000
0               819


### Encontrar o número de avaliações e a classificação média para cada livro

In [26]:
def task_2():
    query = '''
    SELECT 
        book_id,
        COUNT(rating_id) AS num_ratings,
        AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id
    ORDER BY num_ratings DESC;
    '''
    result = pd.read_sql(query, con=engine)
    return result

# Executando a tarefa 2 e mostrando o resultado
result_2 = task_2()
print(result_2.head())

   book_id  num_ratings  avg_rating
0      948          160    3.662500
1      750           88    4.125000
2      673           86    3.825581
3       75           84    3.678571
4      302           82    4.414634


### Identificar a editora que lançou o maior número de livros com mais de 50 páginas

In [25]:
def task_3():
    query = '''
    SELECT 
        p.publisher,
        COUNT(b.book_id) AS books_count
    FROM books b
    JOIN publishers p ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY books_count DESC
    LIMIT 1;
    '''
    result = pd.read_sql(query, con=engine)
    return result

# Executando a tarefa 3 e mostrando o resultado
result_3 = task_3()
print(result_3)

       publisher  books_count
0  Penguin Books           42


### Identificar o autor com a maior média de classificação de livros (apenas para livros com pelo menos 50 classificações)

In [28]:
def task_4():
    query = '''
    SELECT 
        a.author,
        AVG(r.rating) AS avg_author_rating
    FROM books b
    JOIN authors a ON b.author_id = a.author_id
    JOIN ratings r ON b.book_id = r.book_id
    GROUP BY a.author
    HAVING COUNT(r.rating_id) >= 50
    ORDER BY avg_author_rating DESC
    LIMIT 1;
    '''
    result = pd.read_sql(query, con=engine)
    return result

# Executando a tarefa 4 e mostrando o resultado
result_4 = task_4()
print(result_4)

           author  avg_author_rating
0  Diana Gabaldon                4.3


### Encontrar o número médio de avaliações entre usuários que classificaram mais de 50 livros

In [29]:
def task_5():
    query = '''
    SELECT 
        AVG(user_ratings.num_ratings) AS avg_ratings_per_user
    FROM (
        SELECT 
            username,
            COUNT(rating_id) AS num_ratings
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 50
    ) AS user_ratings;
    '''
    result = pd.read_sql(query, con=engine)
    return result

# Executando a tarefa 5 e mostrando o resultado
result_5 = task_5()
print(result_5)

   avg_ratings_per_user
0             54.333333


## Resultados de Cada Consulta

>**Tarefa 1 - Número de livros lançados após 1 de janeiro de 2000:**
* Resultado: 819 livros.

* Conclusão: O número de livros lançados no século XXI é considerável, o que sugere que o catálogo da plataforma é atualizado e abrange obras contemporâneas.

>**Tarefa 2 - Número de avaliações e classificação média para cada livro:**
* Resultado: Exibe o número de avaliações e a média de classificação para os livros.

* Conclusão: O livro mais avaliado possui 160 classificações, e a média das classificações pode ajudar a recomendar livros de maior interesse.

>**Tarefa 3 - Editora com mais livros com mais de 50 páginas:**
* Resultado: "Penguin Books" com 42 livros.

* Conclusão: A Penguin Books se destaca como uma editora de publicações substanciais, com uma quantidade significativa de livros mais longos, possivelmente de maior profundidade.

>**Tarefa 4 - Autor com a maior média de classificação:**
* Resultado: Diana Gabaldon com uma média de 4.3.

* Conclusão: Diana Gabaldon tem uma excelente recepção do público, com uma média de avaliação muito alta, sugerindo um grande apelo entre os leitores.

>**Tarefa 5 - Número médio de avaliações entre usuários que classificaram mais de 50 livros:**
* Resultado: 54.33 avaliações por usuário.

* Conclusão: Usuários altamente engajados fazem em média 54 avaliações, e focar nesses usuários pode ajudar a melhorar a interação e o feedback na plataforma.